In [ ]:
# Import libraries
import numpy as np
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import statistics

In [10]:
# # Download data (run only once)
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
# !unzip data.zip -d data

In [11]:
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [12]:
# Define the CNN model
class HairTypeCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Conv layer: input channels=3, output channels=32, kernel=3x3
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )

        # MaxPool layer: reduces H,W by factor of 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # After conv + pool, we flatten and feed into FC
        # We must compute the flattened size:
        # Input: (3, 200, 200)
        # After conv1 (kernel=3, no padding): (32, 198, 198)
        # After maxpool (2x2): (32, 99, 99)
        flattened_size = 32 * 99 * 99  

        self.fc1 = nn.Linear(flattened_size, 64)
        self.fc2 = nn.Linear(64, 1)  # Single output for binary classification

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # raw logits
        return x


In [13]:
# Calculate total number of parameters
model = HairTypeCNN()

total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

Total number of parameters: 20073473


In [14]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [15]:
# Define data transformations
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = train_transforms 

In [16]:
# Load datasets
train_dataset = datasets.ImageFolder("data/data/train", transform=train_transforms)
validation_dataset = datasets.ImageFolder("data/data/test", transform=test_transforms)

In [17]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

In [18]:
# Define the CNN model
class HairTypeCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)

        # compute flattened size: (3,200,200)->conv->(32,198,198)->pool->(32,99,99)
        flattened = 32 * 99 * 99

        self.fc1 = nn.Linear(flattened, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # raw logits
        return x

model = HairTypeCNN().to(device)

In [19]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [20]:
# Training loop
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val

    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6654, Acc: 0.6262, Val Loss: 0.6102, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5479, Acc: 0.7087, Val Loss: 0.6303, Val Acc: 0.6468
Epoch 3/10, Loss: 0.4858, Acc: 0.7638, Val Loss: 0.6991, Val Acc: 0.5970
Epoch 4/10, Loss: 0.4806, Acc: 0.7650, Val Loss: 0.6097, Val Acc: 0.7015
Epoch 5/10, Loss: 0.4403, Acc: 0.7863, Val Loss: 0.6215, Val Acc: 0.6517
Epoch 6/10, Loss: 0.3493, Acc: 0.8500, Val Loss: 0.6453, Val Acc: 0.6716
Epoch 7/10, Loss: 0.3005, Acc: 0.8775, Val Loss: 0.6953, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2746, Acc: 0.8938, Val Loss: 0.6370, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2119, Acc: 0.9225, Val Loss: 0.6828, Val Acc: 0.7114
Epoch 10/10, Loss: 0.1419, Acc: 0.9525, Val Loss: 0.8263, Val Acc: 0.6965


In [ ]:
# Calculate median training accuracy across epochs
median_acc = statistics.median(history['acc'])
print("Median training accuracy across 10 epochs:", median_acc)


Median training accuracy across 10 epochs: 0.818125


In [22]:
# Calculate standard deviation of training loss across epochs
train_loss_std = statistics.stdev(history['loss'])
print("Standard deviation of training loss:", train_loss_std)

Standard deviation of training loss: 0.1622458634918277


In [24]:
# Augmented transforms ONLY for training set
train_transforms_aug = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Validation / test transforms stay the same as before
val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Re-create datasets with new transforms
train_dataset_aug = datasets.ImageFolder("data/data/train", transform=train_transforms_aug)
validation_dataset = datasets.ImageFolder("data/data/test", transform=val_transforms)

# DataLoaders (same batch size, shuffle settings)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)


In [25]:
# Re-initialize the model, loss function, and optimizer
num_epochs_aug = 10

history_aug = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    history_aug['loss'].append(epoch_loss)
    history_aug['acc'].append(epoch_acc)

    # Validation (test)
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val

    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs_aug}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

# Mean TEST (validation) loss over these 10 epochs
mean_test_loss = sum(history_aug['val_loss']) / len(history_aug['val_loss'])
print("Mean test (validation) loss over augmented training:", mean_test_loss)


Epoch 1/10, Loss: 0.6602, Acc: 0.6613, Val Loss: 0.7089, Val Acc: 0.6866
Epoch 2/10, Loss: 0.5547, Acc: 0.7163, Val Loss: 0.6009, Val Acc: 0.7065
Epoch 3/10, Loss: 0.5345, Acc: 0.7212, Val Loss: 0.5508, Val Acc: 0.7363
Epoch 4/10, Loss: 0.5000, Acc: 0.7600, Val Loss: 0.5279, Val Acc: 0.7463
Epoch 5/10, Loss: 0.4832, Acc: 0.7700, Val Loss: 0.5842, Val Acc: 0.7114
Epoch 6/10, Loss: 0.4704, Acc: 0.7825, Val Loss: 0.5259, Val Acc: 0.7114
Epoch 7/10, Loss: 0.4545, Acc: 0.7775, Val Loss: 0.5017, Val Acc: 0.7910
Epoch 8/10, Loss: 0.4481, Acc: 0.7887, Val Loss: 0.5842, Val Acc: 0.6816
Epoch 9/10, Loss: 0.4281, Acc: 0.8013, Val Loss: 0.4793, Val Acc: 0.7711
Epoch 10/10, Loss: 0.4276, Acc: 0.7975, Val Loss: 0.4887, Val Acc: 0.7562

Mean test (validation) loss over augmented training: 0.555260964827751


In [26]:
# Validation accuracy for last 5 epochs and their average
last5 = history_aug['val_acc'][5:]   # accuracies for epochs 6–10
avg_last5 = statistics.mean(last5)

print("Validation accuracy for last 5 epochs:", last5)
print("Average test accuracy (epochs 6–10):", avg_last5)


Validation accuracy for last 5 epochs: [0.7114427860696517, 0.7910447761194029, 0.681592039800995, 0.7711442786069652, 0.7562189054726368]
Average test accuracy (epochs 6–10): 0.7422885572139304
